# Digitale Nachlässe

Zunächst müssen die notwendigen Bibliotheken geladen und importiert werden.

* `os`: Diese Bibliothek wird verwendet, um mit dem Betriebssystem zu interagieren, z.B. um Dateien und Verzeichnisse zu durchsuchen.
* `pandas`: Eine leistungsstarke Bibliothek zur Datenanalyse und -manipulation.
* `datetime`: Wird verwendet, um Datums- und Zeitinformationen zu verarbeiten.
* `sys`: Ermöglicht den Zugriff auf System-spezifische Parameter und Funktionen.
* `requests`: Eine Bibliothek zum Senden von HTTP-Anfragen.
* `BeautifulSoup`: Ein Werkzeug zum Parsen von HTML- und XML-Dokumenten.

Um die Bibliotheken zunächst zu installieren, kann das Kokmmentar in der nächsten Zeile entfernt werden. Dann einfach die Zelle ausführen. 

In [1]:
# !pip install pandas requests beautifulsoup4 lxml openpyxl

In [2]:
import os
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup

## Funktion zum Sammeln der Informationen

Die folgende Funktion durchsucht ein angegebenes Verzeichnis (`directory_path`) und sammelt Informationen über jede Datei darin. Sie speichert diese Informationen in einer Liste von Wörterbüchern namens `file_info_list` und gibt dieses als Rückgabewert zurück. Für jede Datei legt sie ein eigenes Wörterbuch `file_info` an.

In [3]:
def get_file_info(directory_path):
    file_info_list = []
    for root, dirs, files in os.walk(directory_path):
        for i, file in enumerate(files, start=1):
            file_path = os.path.join(root, file)
            file_stats = os.stat(file_path)
            file_extension = os.path.splitext(file)[1]
            #file_type = get_file_type(file_path)
            file_info = {
                "Laufende Nummer": i,
                "Dateiname": file,
                "Dateiendung": file_extension,
                "Dateipfad": file_path,
                "Änderungsdatum": datetime.fromtimestamp(file_stats.st_mtime).strftime('%Y-%m-%d %H:%M:%S'),
                "Erstellungsdatum": datetime.fromtimestamp(file_stats.st_ctime).strftime('%Y-%m-%d %H:%M:%S'),
                "Dateigröße (Bytes)": file_stats.st_size,
                #"Dateityp": file_type,
                "Bemerkungen": ""
            }
            file_info_list.append(file_info)
    return file_info_list

Falls ein [FITS-Server](https://projects.iq.harvard.edu/fits/fits-web-service) zur Verfügung steht, können die Kommentare in den beiden auskommentierten Zeilen wieder entfernt werden.

## Funktion zum Abrufen des Dateityps

Die nächste Funktion sendet eine Datei an einen FITS-Server, der den Dateityp bestimmt, und gibt diesen zurück. Wenn der Dienst den Dateityp nicht bestimmen kann, wird "Unbekannt" zurückgegeben. Die Funktion wird nicht benötigt, wenn kein FITS-Server zur Verfügung steht.

In [4]:
def get_file_type(file_path):
    url = "http://etc.dnb.de.appr/fits/examine" # Nur im Netz der DNB erreichbar
    with open(file_path, 'rb') as file:
        response = requests.post(url, files={'datafile': file}, verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml-xml')
        format_element = soup.find('identity', format=True) # Durchsucht das XML nach der Angabe des Datetyps
        if format_element:
            return format_element['format']
    return "Unbekannt"

## Angaben sammeln und speichern

In welchem Verzeichnis sollen die Metadaten der Dateien bestimmt werden? Kommentar entfernen und den Pfad angeben.

In [5]:
# directory_path = "path/to/your/directory"
directory_path = "/home/ramon/Downloads"

Als nächstes wird die Liste mit den Angaben zu den Dateien erstellt, in dem die oben definierte Funktion aufgerufen wird:

In [6]:
file_info_list = get_file_info(directory_path)

Diese Liste mit Wörterbüchern ist die Grundlage für einen Dataframe:

In [7]:
df = pd.DataFrame(file_info_list)
df

,Laufende Nummer,Dateiname,Dateiendung,Dateipfad,Änderungsdatum,Erstellungsdatum,Dateigröße (Bytes),Bemerkungen
0,1,SourceCodePro.zip,.zip,/home/ramon/Downloads/SourceCodePro.zip,2024-05-31 18:06:17,2024-05-31 18:06:17,50842511,
1,2,FiraCode.zip,.zip,/home/ramon/Downloads/FiraCode.zip,2024-05-31 18:07:50,2024-05-31 18:07:50,24978177,
2,3,Obsidian-1.6.3-arm64.AppImage,.AppImage,/home/ramon/Downloads/Obsidian-1.6.3-arm64.App...,2024-06-10 20:42:44,2024-06-10 20:42:44,113375411,
3,4,Obisidian.png,.png,/home/ramon/Downloads/Obisidian.png,2024-06-11 19:03:38,2024-06-11 19:03:53,43530,
4,5,install_log.txt,.txt,/home/ramon/Downloads/install_log.txt,2024-06-07 19:00:02,2024-06-07 19:00:02,296,
5,6,catppuccin_mocha.theme,.theme,/home/ramon/Downloads/catppuccin_mocha.theme,2024-05-31 20:35:26,2024-05-31 20:35:26,2277,
6,7,setup_raspi5.sh,.sh,/home/ramon/Downloads/setup_raspi5.sh,2024-06-12 11:16:19,2024-06-12 11:16:19,5986,
7,8,JetBrainsMono.zip,.zip,/home/ramon/Downloads/JetBrainsMono.zip,2024-06-12 11:03:38,2024-06-12 11:03:39,117875043,
8,9,setup_raspi5_II.sh,.sh,/home/ramon/Downloads/setup_raspi5_II.sh,2024-06-12 11:17:35,2024-06-12 11:17:35,1484,
9,10,lazygit,,/home/ramon/Downloads/lazygit,2024-05-19 12:19:58,2024-06-07 21:47:39,17039512,


Zum Speichern des Dataframes kann eine Excel-Datei erstellt werden:

In [8]:
output_file = "output.xlsx"
df.to_excel(output_file, index=False)